In [ ]:
import numpy as np
import polars as pl
from plotnine import ggplot, aes, geom_histogram, ggtitle, theme_bw
import pandas as pd
import json
import utils.constants as constants
from utils.utils import read_blast_output, convert_df_to_gff
from logging_config import logger



#temporary function used only in this scripts
def find_and_group_overlapping(group):
    intervals = []
    strand = group["strand"][0]
    current_start, current_end = group['new_start'][0], group['new_end'][0]
    for i in range(1, len(group)):
        if group['new_start'][i] <= current_end:
            # Overlapping intervals, extend the current interval
            current_end = max(current_end, group['new_end'][i])
        else:
            # Non-overlapping interval, start a new interval
            intervals.append((current_start, current_end))
            current_start, current_end = group['new_start'][i], group['new_end'][i]

    intervals.append((current_start, current_end))
    return pl.DataFrame({
        'seqnames': [group['subject'][0]] * len(intervals),
        'source': ["SatXplor"] * len(intervals),
        'feature' : [group['query'][0]] * len(intervals),
        'start': [start for start, _ in intervals],
        'end': [end for _, end in intervals],
        'score': [100] * len(intervals),
        'strand' : [strand] * len(intervals),
        'frame' : ["."] * len(intervals),
        'group' : [group['query'][0]] * len(intervals),
    })
